In [ ]:

# --- Data Generation ---

# Parameters
num_qubits = 6
matrix_size = 2**num_qubits
num_repetitions = 1000
Mz_values = np.zeros((num_repetitions, matrix_size))


@jit(nopython=True, parallel=True)
eigenvalues = []
eigenvectors = []

for repetition in range(num_repetitions):


    #create random numbers for H.
    np.random.seed(repetition)
    J = np.random.uniform(low=-1, high=0, size=(num_qubits, num_qubits))
    for i in range(num_qubits):
        for j in range(num_qubits):
            if i >= j:
                J[i, j] = 0
    J = (J + J.T)

    np.random.seed(repetition +10)
    K = np.random.uniform(low=-1, high=1, size=(num_qubits, num_qubits))
    for i in range(num_qubits):
        for j in range(num_qubits):
            if i >= j:
                K[i, j] = 0
    K = (K + K.T)

    np.random.seed(repetition +20)
    h = np.random.uniform(low=-0.04, high=0.04)

    np.random.seed(repetition +30)
    g = np.random.uniform(low=-6, high=6, size=num_qubits)




    # create the matrix of H.
    matrix = np.zeros((matrix_size, matrix_size))
    for i in range(matrix_size):
        for j in range(matrix_size):
            H_value = 0
            for i1 in range(num_qubits):
                for j1 in range(num_qubits):
                    if j == i ^ (2 ** i1 + 2 ** j1):
#This part adds contributions to H_value based on the coupling constants K and the XOR operation (^).
#It checks if the bit-flipped indices i ^ (2 ** i1 + 2 ** j1) are equal to j and adds the corresponding K valeu.
                        H_value += K[i1, j1]
                    if j == i:
                        sign = 1
                        if (i & 2 ** i1) != 0:# & returns 1 if both the bits are 1, otherwise 0.
                            sign = -sign
                        if (i & 2 ** j1) != 0:
                            sign = -sign

                        H_value += sign * J[i1, j1]

            for i1 in range(num_qubits):
                if j == i ^ (2 ** i1):# external fields g for bit-flipped indices.
                    H_value += g[i1]

            if j == i:
                H_value += h # external field h when j is equal to i.
            matrix[i, j] = H_value




    # calculate eigenvalues and eigenvectors.
    eigenval, eigenvect = np.linalg.eigh(matrix)
   # print(eigenval)
   # print(eigenvect)
    min_eigenval = np.min(eigenval)
    min_eigenvec = eigenvect[np.argmin(eigenval), :]

    eigenvalues.append(min_eigenval)
    eigenvectors.append(min_eigenvec)

    #save the eigenvectors as a CSV file
eigenvectors_csv = np.array(eigenvectors)
np.savetxt('eigenvectors.csv', eigenvectors_csv, delimiter=',')
    #print
for repetition in range(num_repetitions):
    print("Repetition", repetition+1)
    print("Eigenvalue:", eigenvalues[repetition])
    print("Eigenvector:", eigenvectors[repetition])

    print()

  #creating the matrix of data, eigenvectors. each column is eigenvector of a certain H.
eigenvectors_matrix = np.column_stack(eigenvectors)

print("Eigenvectors Matrix:")
print(eigenvectors_matrix)


Mzt = []
for column in eigenvectors_matrix.T:
    i = 0
    Mz = 0
    for component in column:
        for n_prime in range(num_qubits):
            if (2 ** n_prime) & i != 0 :
                Mz += abs(component) ** 2  / num_qubits
          #Mz += abs(component) ** 2 * (2 * ((2 ** n_prime) & i) - 1) / num_qubits
        i += 1
    Mz=2*Mz-1
    Mzt.append(Mz)

Mzt_row = np.array(Mzt)
print("Mzt:", Mzt,"\n")
print(len(Mzt),"\n")
print(Mz)
